In [2]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Conv2D, Dropout, BatchNormalization, MaxPooling2D, Flatten, Input
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


In [3]:
cifar10 = tf.keras.datasets.cifar10
(X_train,y_train), (X_test, y_test) = cifar10.load_data()
X_train, X_test = X_train/255.0, X_test/255.0
y_train, y_test = y_train.flatten(), y_test.flatten()
print(X_train.shape)

(50000, 32, 32, 3)


In [4]:
k = len(set(y_train))
print(k)

10


In [5]:
class_names = ['Plane', 'Car', 'Bird', 'Cat', 'Deer', 'Dog', 'Frog', 'Horse', 'Ship', 'Truck']

In [6]:
i = Input(X_train[0].shape)


x = Conv2D(32, (3,3), activation = 'relu', padding = 'same')(i)
x = BatchNormalization()(x)
x = Conv2D(32, (3,3), activation = 'relu', padding = 'same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2,2))(x)

x = Conv2D(64, (3,3), activation = 'relu', padding = 'same')(i)
x = BatchNormalization()(x)
x = Conv2D(64, (3,3), activation = 'relu', padding = 'same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2,2))(x)

x = Conv2D(128, (3,3), activation = 'relu', padding = 'same')(i)
x = BatchNormalization()(x)
x = Conv2D(128, (3,3), activation = 'relu', padding = 'same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2,2))(x)

x = Flatten()(x)
x = Dropout(0.2)(x)
x = Dense(1024, activation = 'relu')(x)
x = Dropout(0.2)(x)
x = Dense(k,activation = 'softmax')(x)


model = Model(i,x)

In [7]:
!pip install python-telegram-bot==13.15

In [8]:
!pip show python-telegram-bot


Name: python-telegram-bot
Version: 13.15
Summary: We have made you a wrapper you can't refuse
Home-page: https://python-telegram-bot.org/
Author: Leandro Toledo
Author-email: devs@python-telegram-bot.org
License: LGPLv3
Location: C:\Users\mv350\anaconda3\Lib\site-packages
Requires: APScheduler, cachetools, certifi, pytz, tornado
Required-by: 


In [9]:
from telegram.ext import *
from io import BytesIO
import cv2
import tensorflow as tf
import numpy as np

In [10]:
TOKEN = "7515667266:AAFJgXs8Tyvxhy_cxEikfLjzUiEfRcP8haM"
    

In [ ]:
from telegram.ext import Updater, CommandHandler
from queue import Queue





def start(update, context):
    update.message.reply_text("hello welcome!")
    
    
def help(update, context):
    update.message.reply_text("""
    /start - starts conversation
    /help - shows this message
    /train - trains the bot
    """)
    
def train(update, context):
    update.message.reply_text("Model is Being Trained...")
    model.compile(optimizer = 'adam',
             loss = 'sparse_categorical_crossentropy',
             metrics = ['accuracy'])
    model.fit(X_train,y_train, validation_data = (X_test,y_test), epochs =10)
    model.save('Cifar10_Classifier.model')
    update.message.reply_text("Done you can now send a photo!") 
    
def handle_message(update,context):
    update.message.reply_text("please train the model and send a photo")
    
    
    
def handle_photo(update, context):
    file = context.bot.get_file(update.message.photo[-1].file_id)
    f = BytesIO(file.download_as_bytearray())
    file_bytes = np.asarray(bytearray(f.read()), dtype = np.uint8)
    
    
    img = cv2.imdecode(file_bytes, cv2.IMREAD_COLOR)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    img = cv2.resize(img, (32,32), interpolation = cv2.INTER_AREA)
    
    
    prediction = model.predict(np.array([img/ 255.0]))
    update.message.reply_text(f"In this Image I see a...{class_names[np.argmax(prediction)]}")
    
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters


TOKEN = #Type your Token here in "" made from 'botfather' bot in telegram(note:keep your token secret)


updater = Updater(TOKEN, use_context=True)
dp = updater.dispatcher




dp.add_handler(CommandHandler("start", start))
dp.add_handler(CommandHandler("train", train))
dp.add_handler(CommandHandler("help", help))
dp.add_handler(MessageHandler(Filters.text & ~Filters.command, handle_message))  # Handles non-command text messages
dp.add_handler(MessageHandler(Filters.photo, handle_photo))  # Handles photo messages

# Start the bot
updater.start_polling()
updater.idle()
